In [8]:
# %pip install numpy as np
# %pip install pandas as pd
# %pip install scikit-learn
# %pip install matplotlib
%pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
# Import the libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
customer_df = pd.read_csv("datasets/Retail_Store_Consumer_NoTech.csv",encoding="latin-1")

In [49]:
customer_df['shipment_date'] = pd.to_datetime(customer_df['shipping date (DateOrders)'])
customer_df['order_date'] = pd.to_datetime(customer_df['order date (DateOrders)'])

customer_df.head(5)

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Region,Order State,Order Status,Product Name,shipping date (DateOrders),Shipping Mode,order_date,year,month,shipment_date
0,CASH,2,1,78.660004,262.200012,Late delivery,1,73,Sporting Goods,Caguas,...,Southeast Asia,Yakarta,CLOSED,Smart watch,12/6/2017 14:37,First Class,2017-12-04 14:37:00,2017,12,2017-12-06 14:37:00
1,CASH,3,4,109.669998,324.470001,Advance shipping,0,73,Sporting Goods,Tampa,...,Southeast Asia,Yakarta,CLOSED,Smart watch,12/7/2017 13:34,Standard Class,2017-12-04 13:34:00,2017,12,2017-12-07 13:34:00
2,CASH,2,2,136.509995,278.589996,Shipping on time,0,73,Sporting Goods,Irvington,...,Southeast Asia,RangÃºn,CLOSED,Smart watch,12/6/2017 9:43,Second Class,2017-12-04 09:43:00,2017,12,2017-12-06 09:43:00
3,CASH,2,1,23.650000,268.760010,Late delivery,1,73,Sporting Goods,Gardena,...,Southeast Asia,RangÃºn,CLOSED,Smart watch,12/6/2017 8:39,First Class,2017-12-04 08:39:00,2017,12,2017-12-06 08:39:00
4,CASH,2,4,158.990005,324.470001,Advance shipping,0,73,Sporting Goods,Caguas,...,Southeast Asia,Capital Nacional,CLOSED,Smart watch,12/6/2017 0:57,Standard Class,2017-12-04 00:57:00,2017,12,2017-12-06 00:57:00


In [50]:
customer_df.info

<bound method DataFrame.info of       Type  Days for shipping (real)  Days for shipment (scheduled)  \
0     CASH                         2                              1   
1     CASH                         3                              4   
2     CASH                         2                              2   
3     CASH                         2                              1   
4     CASH                         2                              4   
...    ...                       ...                            ...   
9771  CASH                         3                              2   
9772  CASH                         3                              2   
9773  CASH                         2                              1   
9774  CASH                         2                              4   
9775  CASH                         3                              4   

      Benefit per order  Sales per customer   Delivery Status  \
0             78.660004          262.200012     La

Feature Engineering

In [51]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

def feature_engineering(customer_df):
    # Create a copy of the dataframe
    customer_df = customer_df.copy()

    # Debug: Check if the input DataFrame contains the necessary columns
    print("Input DataFrame columns:", customer_df.columns)

    # Ensure that necessary columns exist
    if 'Delivery Status' not in customer_df.columns or 'Order Region' not in customer_df.columns:
        raise ValueError("Missing required columns: 'Delivery Status' or 'Order Region'")

    # Ordinal encode the 'Delivery Status' feature
    oe = OrdinalEncoder(categories=[['Late delivery', 'Shipping on time', 'Advance shipping']])
    delivery_status = oe.fit_transform(customer_df[['Delivery Status']])
    delivery_status_col = ['Delivery Status_encoded']
    delivery_status_df = pd.DataFrame(delivery_status, columns=delivery_status_col)
    
    # Debug: Print the head of the encoded Delivery Status
    print("Encoded Delivery Status:\n", delivery_status_df.head())

    # One-hot encode the 'Order Region' feature
    ohe = OneHotEncoder(sparse_output=False)
    order_region = ohe.fit_transform(customer_df[['Order Region']])
    order_region_cols = ohe.get_feature_names_out(['Order Region'])
    order_region_df = pd.DataFrame(order_region, columns=order_region_cols)
    
    # Debug: Print the head of the encoded Order Region
    print("Encoded Order Region:\n", order_region_df.head())

    # Concatenate the encoded features with the original dataframe
    customer_df = pd.concat([customer_df, delivery_status_df, order_region_df], axis=1)
    
    # Debug: Check the DataFrame columns after concatenation
    print("Columns after concatenation:", customer_df.columns)

    # Drop the original categorical features
    customer_df = customer_df.drop(['Delivery Status', 'Order Region'], axis=1)

    # Debug: Check the DataFrame after dropping the original columns
    print("Columns after dropping original categorical columns:", customer_df.columns)

    # Create new feature: 'days_bwt_real_sch_shipping_df'
    if 'Days for shipping (real)' in customer_df.columns and 'Days for shipment (scheduled)' in customer_df.columns:
        customer_df['days_bwt_real_sch_shipping_df'] = customer_df['Days for shipping (real)'] - customer_df['Days for shipment (scheduled)']
    else:
        raise ValueError("Missing required columns for new feature creation: 'Days for shipping (real)' or 'Days for shipment (scheduled)'")

    # Debug: Check the final DataFrame after feature creation
    print("Final DataFrame:\n", customer_df.head())

    # Return the updated DataFrame
    return customer_df

# Example usage:
# Assuming customer_df is your input DataFrame.
# If you're running this in a script, you can load your data like:
# customer_df = pd.read_csv('your_data.csv')

customer_df = feature_engineering(customer_df)


Input DataFrame columns: Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Id', 'Customer Segment', 'Customer State',
       'Customer Street', 'Department Id', 'Department Name', 'Latitude',
       'Longitude', 'Market', 'Order City', 'Order Country',
       'order date (DateOrders)', 'Order Item Discount',
       'Order Item Discount Rate', 'Order Item Product Price',
       'Order Item Profit Ratio', 'Order Item Quantity', 'Sales',
       'Order Region', 'Order State', 'Order Status', 'Product Name',
       'shipping date (DateOrders)', 'Shipping Mode', 'order_date', 'year',
       'month', 'shipment_date'],
      dtype='object')
Encoded Delivery Status:
    Delivery Status_encoded
0                      0.0
1                      2.0
2                      1.0
3              

In [ ]:
# Define the function to train and evaluate the model
def train_and_evaluate_model(customer_df):
  # Create a copy of the dataframe
  customer_df = customer_df.copy()
  
  # Generate some random discount types for the target variable
  customer_df['discount'] = np.random.choice(['percentage', 'fixed', 'free shipping', 'bundled'], size=len(customer_df))
  
  # Split the dataframe into features and target
  X = customer_df.drop('discount_type', axis=1)
  y = customer_df['discount_type']
  
  # Split the data into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  
  # Define a list of models to compare
  models = [LogisticRegression(), DecisionTreeClassifier(), RandomForestClassifier(), SVC()]
  model_names = ['Logistic Regression', 'Decision Tree', 'Random Forest', 'Support Vector Machine']
  
  # Loop over the models and compare their performance
  for i, model in enumerate(models):
    # Print the model name
    print(f'Model: {model_names[i]}')
    
    # Perform cross-validation on the train set
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    print(f'Cross-validation scores: {cv_scores}')
    print(f'Mean cross-validation score: {cv_scores.mean()}')
    
    # Fit the model on the train set
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate the model on the test set
    test_score = accuracy_score(y_test, y_pred)
    print(f'Test score: {test_score}')
    
    # Plot the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['percentage', 'fixed', 'free shipping', 'bundled'], yticklabels=['percentage', 'fixed', 'free shipping', 'bundled'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion matrix for {model_names[i]}')
    plt.show()
    
    # Print the classification report
    print(classification_report(y_test, y_pred))
    
    # Print a blank line
    print()

# Generate 1000 customer data and apply feature engineering to it
customer_df = feature_engineering(generate_customer_data(1000))

# Call the function to train and evaluate the model
train_and_evaluate_model(customer_df)

In [52]:
customer_df.columns

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Late_delivery_risk',
       'Category Id', 'Category Name', 'Customer City', 'Customer Country',
       'Customer Id', 'Customer Segment', 'Customer State', 'Customer Street',
       'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market',
       'Order City', 'Order Country', 'order date (DateOrders)',
       'Order Item Discount', 'Order Item Discount Rate',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order State', 'Order Status',
       'Product Name', 'shipping date (DateOrders)', 'Shipping Mode',
       'order_date', 'year', 'month', 'shipment_date',
       'Delivery Status_encoded', 'Order Region_Canada',
       'Order Region_Caribbean', 'Order Region_Central Africa',
       'Order Region_Central America', 'Order Region_Central Asia',
       'Order Region_East Africa', 'Order Region_E